In [264]:
# install if needed
!pip install searchtweets


/bin/bash: pip: command not found


In [265]:
import json
from searchtweets import load_credentials, gen_rule_payload, ResultStream

In [266]:
filen = "_01_07_21_VIII_US"
scope = 'US Data/'
#scope = 'NY Data/'
path = "./" + scope
geocode = "39.8,-95.583068847656,2500km" if scope == 'US Data/' else "40.714353,-74.00597299999998,20km"
FILENAME = path + "data" + filen + ".jsonl"

# Search terms
search_words = "asians OR asian"
date_since = "2021-01-07"
date_to = "2021-01-08" # excluded

In [267]:
#change API scope
#full archived scope = full archived
#thirty scope = 30day
API_SCOPE = '30day'

In [268]:
rule = gen_rule_payload(search_words,
                        results_per_call=100, # 100 for sandbox, 500 for paid 
                        from_date=date_since,
                        to_date=date_to,
                       )


In [269]:
API_KEY = 'cENP2J4B2hqeJEuiTNWAwKmOu' # consumer key
API_SECRET_KEY = 'InNdy2u7HjnI9JrEz4gLP8i6Ro1t3R6fuVV2JhXMb5nEN80wuk' # consumer secret
DEV_ENVIRONMENT_LABEL = 'Research'

In [270]:
import yaml
config = dict(
    search_tweets_api=dict(
        account_type='premium',
        endpoint=f"https://api.twitter.com/1.1/tweets/search/{API_SCOPE}/{DEV_ENVIRONMENT_LABEL}.json",
        consumer_key=API_KEY,
        consumer_secret=API_SECRET_KEY
    )
)
with open('twitter_keys.yaml', 'w') as config_file:
    yaml.dump(config, config_file, default_flow_style=False)

In [271]:
premium_search_args = load_credentials("twitter_keys.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)

Grabbing bearer token from OAUTH


In [272]:
rs = ResultStream(rule_payload=rule,
                  max_results=1000,
                  **premium_search_args)

In [273]:
rs

ResultStream: 
	{
    "username":null,
    "endpoint":"https:\/\/api.twitter.com\/1.1\/tweets\/search\/30day\/Research.json",
    "rule_payload":{
        "query":"asians OR asian",
        "maxResults":100,
        "toDate":"202101080000",
        "fromDate":"202101070000"
    },
    "tweetify":true,
    "max_results":1000
}

In [274]:
#### save the fetched data 
# Script prints an update to the CLI every time it collected another X Tweets
PRINT_AFTER_X = 1000
#PRINT_AFTER_X = 100
with open(FILENAME, 'a', encoding='utf-8') as f:
    n = 0
    for tweet in rs.stream():
        n += 1
        if n % PRINT_AFTER_X == 0:
            print('{0}: {1}'.format(str(n), tweet['created_at']))
        json.dump(tweet, f)
        f.write('\n')
print('done')

1000: Thu Jan 07 23:34:46 +0000 2021
done


In [275]:
#### read the save data

import pandas as pd
import numpy as np

df = pd.read_json(FILENAME, lines=True)
df.head()


,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,matching_rules,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,display_text_range,possibly_sensitive,extended_entities
0,2021-01-07 23:59:58+00:00,1347332179777478658,1347332179777478656,RT @gayman42069: Pogchamp got cancelled becaus...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-07 23:59:55+00:00,1347332166661726208,1347332166661726208,RT @noemicaseres53: @ALDubFTnQT @cpjayloni @we...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,in,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-07 23:59:53+00:00,1347332159938301952,1347332159938301952,RT @brightwin_fess: good morning /bw s☀️ yang ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,in,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-07 23:59:52+00:00,1347332153067986944,1347332153067986944,RT @Tookta92166405: #HaveABrightDay \nสวัสดีเช...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,th,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-07 23:59:47+00:00,1347332135502360576,1347332135502360576,@SecElaineChao you should have quit when Agent...,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,NaN,8.260659e+17,8.260659e+17,...,en,[{'tag': None}],{'full_text': '@SecElaineChao you should have ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [276]:
df.shape

(1000, 36)